In [1]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta
from itertools import combinations

from local_utils import *
import lightgbm as lgb

%load_ext autoreload
%autoreload 2


In [2]:
test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = "C:\\Users\\Admin\\Desktop\\AIM 2сем\\ML2\\hw2"

In [3]:
product_information = pl.read_csv(
    os.path.join(data_path, 'product_information.csv'),
    ignore_errors=True
)

In [4]:
product_information

product_id,name,brand,type,category_id,category_name,price,discount_price
i64,str,str,str,i64,str,i64,i64
26176363,"""Развивающие тесты (3-4 года) (…","""Machaon""","""Печатная книга: Развитие детей""",780,"""Книги""",380,274
29898500,"""Mexx Туалетная вода Ice Touch …","""Mexx""","""Туалетная вода""",117,"""Мужская""",2645,1859
33967827,"""64 ГБ USB Флеш-накопитель USB …","""SmartBuy""","""USB-флеш-накопитель""",178,"""Флешки и CD-R""",1690,469
135938830,"""Чай листовой чёрный Ahmad Tea …","""Ahmad Tea""","""Чай листовой""",465,"""Чай листовой""",319,244
137920686,"""Seagate 4 ТБ Внешний жесткий д…","""Seagate""","""Внешний жесткий диск""",615,"""Внешние жесткие диски""",28590,9539
…,…,…,…,…,…,…,…
1665110648,"""Салфетки бумажные 1 слоя 24 см…","""Gratias""","""Бумажные салфетки""",301,"""Салфетки бумажные""",0,0
1667861556,"""Зубная паста детская R.O.C.S. …","""R.O.C.S.""","""Зубная паста""",642,"""Зубные пасты и ополаскиватели""",0,0
1674778291,"""Доска разделочная 36x24x1,2см …","""BY""","""Разделочная доска""",202,"""Ножи и доски""",0,0


In [11]:
pi_test = product_information.sample(1000)
pi_test = pi_test.to_pandas()

In [ ]:
product_information = product_information.to_pandas()

In [36]:
product_information['cluster'] = -1

In [47]:
#1
milk = ['кефир', 'сметан', 'масло', 'йогурт', 'морожен', 'яйц', 'молок', 'сливк', 'ряжен', 'простокваш', 'пахт', 'сгущенк', 'сыворот', 'exponenta',
        'молоч', 'творо', 'экспонента'
]
#2
sweet = ['конфет', 'мармелад', 'сироп', 'энергетик', 'кекс', 'сахар', 'шоколад', 'батончик', 'варен', 'печень', 'торт', 'чипс', 'лейс', 'lays', 'халв', 
         'зефир', 'пастил', 'пряник', 'леден', 'драже', 'карамел', 'джем', 'мед', 'вафл', 'хлебец', 'семечки', 'киндер', 'хлопья', 'попкорн','сухар',
         'какао', 'слад', 'kinder', 'ролл', 'нори', 'сникерс', 'рождения', 'жвачка', 'bombbar', 'адреналин', 'десерт', 'суш', 
]
#3
cereals = ["греч", "рис", "овсян", "пшен", "перлов", "манк", "кукурузн", "ячнев", "булгур", "киноа", "макарон", "спагетт", "лапш", "вермишел", "фасол", 
           "горох", "нут", "чечевиц", "полб", 'каша', 'доширак', 'геркулес', 'хумус', 'завтрак', 'barilla', 'гранола', 'мюс', 
           
]
#4
veg_and_fruits = ['изюм', 'перец', 'огур', 'помидор', 'шампиньон', 'банан', 'фрукт', 'овощ', 'апельсин', 
                  'томат', 'свекл', 'баклажан', 'мандарин', 'ягод', 'гриб', 'лимон', 'персик', 'цукин', 
                  'яблок', 'груш', 'салат', 'черри', 'морков', 'патиссон', 'картош', 'картоф','капуст',
                  'груша', 'персик', 'апельсин', 'мандарин', 'баклажан', 'лук', 'авокадо', 'клубник', 
                  'зелен', 'кукуруз', 'орех', 'арбуз', 'продукт', 'оливк', 'череш', 'манго', 'шпинат',
                  'голуби', 'виноград', 'брокколи', 'сельдер', 'кабач', 'малин', 'киви', 'редис', 'фисташк', 
                  'лайм', 'маслин', 'семен', 'свеж', 'айсберг', 'азбук', 'арахис', 'fresh', 'финик', 'вишн', 
                  'спарж', 'щавель', 'ананас', 'тыкв', 
]
#5
drink = ['квас', 'сок', 'вод', 'кофе', 'чай', 'лимонад', 'напитк', 'кола', 'пиво', 'напиток', 'лед', 'коктейль'
         'святой', 'источник', 'боржоми', 'морс', 'cola', 'шишкин', 'лес', 'черноголов', 'тоник', 'липтон', 'рычал', 
         'тархун', 'вино', 
]
#6
bread = ['хлеб', 'булк', 'батон', 'багет', 'еда', 'мука', 'пицца', 'лаваш', 'тесто', 'кулич', 'пирожное', 'блин', 'круассан', 'пудинг', 'чиабатта', 
        'сушк', 'пирог', 'пасх', 'рулет', 'выпеч', 'дрожж', 'бургер', 'чизкейк', 
]
#7
cheese = ['сыр', 'брынз', 'моцарел', 'пармезан']
#8
pets = ['корм', 'наполнит', 'лакомств', 'вискас', 'педигри', 'собак', 'кошек', 'кроликов']
#9
meat = ['рыб', 'сосис', 'колбас', 'куриц', 'филе', 'говядин', 'мяс', 'крабов', 'креветк', 'свинин', 'фарш', 'кальмар', 'ветчина', 'пельмени', 'бекон'
        'индейка', 'котлет', 'суп', 'икр', 'груд', 'индейка', 'курин', 'мираторг', 'бекон', 'протеин', 'стейк', 'шашлык', 'тунец', 'форель', 'наггетсы',
        'бедро', 'индилайт', 'нарезк', 'ребр', 'голен', 'консерв', 'лосос', 'фрикадел', 'семг', 'рамен', 'халяль', 'сало', 'сельдь', 'горячая', 'сардель', 
        'тушен', 'онигири', 'свин', 'мидии', 'утк', 
]
#10
household = ['мыл', 'гел', 'паст', 'туалетн', 'крем', 'средств', 'краск', 'стакан', 'волос', 'шампун', 'кондиционер', 'салфетк', 'порошок', 'жидкост', 
             'капсул', 'бумаг', 'бумаж', 'полотен', 'проклад', 'дезодорант', 'батарейки', 'пакет', 'презервативы', 'ватные', 'диски', 'горошек', 'подар',
            'тампоны', 'комаров', 'швабра', 'свечи', 'посуда', 'сковород', 'однораз', 'пищев', 'стол', 'скотч', 'клей', 'книг', 'пласт', 'карт', 'подушк', 
            'отбел', 'чемодан', 'час', 'фольга', 'матрас', 'тетрад', 'доместос', 'удлинитель', 'резинк', 'кукл', 'уголь', 'кружк', 
]

#11
kids = ['подгузник', 'памперс', 'дет', 'соск', 'пюре', 'агуша', 'фрутоняня', 'лего', 'lego', 'игра', 'смес', 'биолакт', 'шарик', 'тема']
#12
clothing = [
    'куртк', 'пальт', 'пуховик', 'ветровк', 'плащ', 'шуб', 'дубленк', 'зимн', 'демисезонн',
    'футболк', 'рубашк', 'блузк', 'водолазк', 'толстовк', 'свитер', 'джемпер', 'кардиган', 'худи',
    'кофт', 'боди', 'топ', 'майк',
    'брюк', 'джинс', 'штан', 'штаны', 'леггинс', 'бридж', 'капри', 'юбк', 'сарафан', 'комбинезон',
    'плать', 'сарафан', 'туник', 'сарафан',
    'трусы', 'трусики', 'бюстгальтер', 'лифчик', 'белье', 'пижам', 'ночн', 'халат', 'комплект',
    'спортивн', 'лосин', 'велосипедк', 'термобелье', 'олимпийк', 'тренировочн', 'спорткостюм',
    'купальник', 'бикини', 'плавк', 'монокини', 'танкини', 'пляжн',
    'шапк', 'кепк', 'шляп', 'панам', 'косынк', 'бандан', 'берет',
    'туфл', 'кроссовк', 'ботинк', 'сапог', 'босоножк', 'сандал', 'угг', 'каблук', 'мокасин', 'лофер',
    'черепик', 'кед', 'балетк', 'сабо', 'дутик',
    'ремен', 'пояс', 'галстук', 'бабочк', 'шарф', 'перчатк', 'варежк', 'зонт', 'очк', 'солнцезащитн',
    'сумк', 'рюкзак', 'клатч', 'кошельк', 'портфель',
    'халат', 'униформа', 'спецовк', 'комбинезон', 'фартук',
    'детск', 'малыш', 'подросток', 'распашонк', 'ползунк', 'пеленк',
    'нос', 'гольф', 'чулк', 'колготк', 'подследник',
    'аксессуар', 'декор', 'украшен', 'бижутери', 'стильн', 'модн', 'брендов', 'одежда', 'шорт', 'легинс'
]
#13
health = [
    # Органы и системы
    'глаз', 'зрен', 'офтальм', 'зрит', 'сетчатк', 'роговиц',
    'сердц', 'карди', 'гипертон', 'давлен', 'пульс', 'аритм',
    'печен', 'гепат', 'желч', 'желуд', 'гастр', 'кишечн', 'панкреат',
    'почк', 'нефр', 'мочевой', 'цистит', 'простат',
    'легк', 'пневм', 'бронх', 'астм', 'кашел', 'насморк', 'горл', 'отит',
    'зуб', 'стомат', 'десн', 'кариес', 'флюс',
    'кож', 'дермат', 'акне', 'псориаз', 'экзем', 'аллерг',
    'нерв', 'неврол', 'головн', 'мигрен', 'инсульт', 'эпилепс',
    'сустав', 'артр', 'остеохондр', 'позвон', 'ревмат', 'артрит',
    'щитовид', 'эндокрин', 'диабет', 'гормон', 'инсулин',
    
    # Препараты и формы выпуска
    'таблетк', 'капсул', 'капл', 'маз', 'крем', 'гел', 'спрей', 'ингалятор',
    'суспенз', 'сироп', 'настойк', 'экстракт', 'назол', 'отинум',
    'антибиот', 'противовирусн', 'антигистаминн', 'анальгин', 'парацетамол',
    'ибупрофен', 'спазмолгон', 'но-шп', 'корвалол', 'валидол',
    'витамин', 'минерал', 'омега', 'магн', 'коллаген', 'пробиот',
    
    # Симптомы и состояния
    'бол', 'воспален', 'отек', 'опухол', 'зуд', 'жжен', 'тошнот', 'рвот',
    'диаре', 'запо', 'метеоризм', 'слабост', 'усталост', 'стресс', 'депресс',
    'бессонн', 'тревожн', 'паническ', 'судорог', 'свист', 'хрип',
    'кровоточивост', 'кров', 'гемоглоб', 'анеми', 'варикоз', 'геморро', 'линзы'
]
#14
sauces = [
    'майонез', 'кетчуп', 'горчиц', 'аджик', 'хрен', 'соус', 'тартар', 'бешамель', 'песто', 'цезар','барбекю', 'чили', 'терияк', 'сальса', 'гуакамоле',
    'сметанн', 'чесночн', 'имбирн', 'устричн', 'рыбн','ткемал', 'кисло-сладк', 'бальзамик', 'вустершир', 'карри','сырн', 'шашлычн', 'гриль', 'паприк', 'чеддер',
    'провансаль', 'брусничн', 'клюквен', 'мятн', 'укропн', 'уксус', 'приправ', 'сода'
]
#15
gadgets = [
    'смартфон', 'iphone', 'андроид', 'телефон', 'кнопочный телефон', 'раскладушка',
    'xiaomi', 'samsung', 'honor', 'huawei', 'nokia', 'realme', 'oppo', 'vivo',
    'планшет', 'ipad', 'электронная книга', 'электронная читалка', 'букридер',
    'ноутбук', 'ультрабук', 'macbook', 'asus', 'lenovo', 'hp', 'dell', 'acer', 'msi',
    'компьютер', 'системный блок', 'моноблок', 'неттоп',
    'клавиатура', 'мышь', 'мышка', 'коврик для мыши', 'трекпад', 'стилус', 'графический планшет',
    'веб-камера', 'микрофон', 'док-станция',
    'процессор', 'видеокарта', 'оперативная память', 'ssd', 'жесткий диск', 'hdd',
    'материнская плата', 'блок питания', 'охлаждение', 'кулер', 'термопаста',
    'наушники', 'airpods', 'гарнитура', 'беспроводные наушники', 'накладные наушники',
    'вкладыши', 'вакуумные', 'колонка', 'динамик', 'bluetooth колонка', 'портативная колонка',
    'микрофон', 'звуковая карта', 'dac', 'усилитель',
    'умные часы', 'smart watch', 'фитнес-браслет', 'умный браслет', 'пульсометр',
    'шагомер', 'gps трекер', 'умное кольцо',
    'чехол', 'защитное стекло', 'плёночка', 'бампер', 'держатель', 'попсокет',
    'кардхолдер', 'кабель', 'зарядка', 'адаптер', 'power bank', 'повербанк',
    'кабель usb', 'кабель type-c', 'lightning', 'переходник', 'разветвитель',
    'подставка', 'держатель для телефона', 'автодержатель',
    'игровая мышь', 'игровая клавиатура', 'джойстик', 'геймпад', 'руль', 'педали',
    'аркадный стик', 'vr', 'виртуальная реальность', 'очки vr', 'гарнитура vr',
    'фотоаппарат', 'зеркалка', 'беззеркалка', 'экшн-камера', 'go pro', 'объектив',
    'штатив', 'монопод', 'стабилизатор', 'гимбал', 'квадрокоптер', 'дрон',
    'умная лампа', 'умная розетка', 'умный дом', 'датчик', 'сенсор', 'камера наблюдения',
    'видеоняня', 'умный замок', 'робот-пылесос', 'умный термостат',
    'принтер', 'сканер', 'ксерокс', 'мфу', 'факс', 'ламинтор', 'брошюровщик',
    'уничтожитель бумаг', 'шредер',
    'электроника', 'гаджет', 'новинка', 'аксессуар', 'комплектующая', 'устройство',
    'беспроводной', 'bluetooth', 'wi-fi', 'usb', 'type-c', 'lightning', 'адаптер', 'вентилятор'
    'паштет', 'айфон', 'увлаж', 'освеж', 'пылесос', 'pro', 'холодильник', 'су вид', 'телевизор', 
    'блендер', 'весы', 
    
]
#18
spices = [
    'перец', 'соль', 'паприк', 'куркум', 'кориандр', 'кардамон', 'имбир', 
    'чеснок', 'лук', 'лавров', 'розмарин', 'тимьян', 'базилик', 'орегано', 
    'шалфей', 'мята', 'укроп', 'петрушк', 'кинз', 'тмин', 'зир', 'кориц', 
    'гвоздик', 'мускат', 'ванил', 'шафран', 'горчиц', 'хрен', 'карри', 
    'прованс', 'фенхел', 'анис', 'кумин', 'чили', 'бадьян', 'кайен',
    'хмел', 'цикор', 'том ям', 
]
#19
woman = ['женск', 'букет']

#20
man = ['мужск']

In [48]:
product_information.loc[product_information['name'].str.contains('|'.join(milk), case=False, na=False), 'cluster'] = 1
print('1 finished')
product_information.loc[product_information['name'].str.contains('|'.join(sweet), case=False, na=False), 'cluster'] = 2
print('2 finished')
product_information.loc[product_information['name'].str.contains('|'.join(cereals), case=False, na=False), 'cluster'] = 3
print('3 finished')
product_information.loc[product_information['name'].str.contains('|'.join(veg_and_fruits), case=False, na=False), 'cluster'] = 4
print('4 finished')
product_information.loc[product_information['name'].str.contains('|'.join(drink), case=False, na=False), 'cluster'] = 5
print('5 finished')
product_information.loc[product_information['name'].str.contains('|'.join(bread), case=False, na=False), 'cluster'] = 6
print('6 finished')
product_information.loc[product_information['name'].str.contains('|'.join(cheese), case=False, na=False), 'cluster'] = 7
print('7 finished')
product_information.loc[product_information['name'].str.contains('|'.join(pets), case=False, na=False), 'cluster'] = 8
print('8 finished')
product_information.loc[product_information['name'].str.contains('|'.join(meat), case=False, na=False), 'cluster'] = 9
print('9 finished')
product_information.loc[product_information['name'].str.contains('|'.join(household), case=False, na=False), 'cluster'] = 10
print('10 finished')
product_information.loc[product_information['name'].str.contains('|'.join(kids), case=False, na=False), 'cluster'] = 11
print('11 finished')
product_information.loc[product_information['name'].str.contains('|'.join(clothing), case=False, na=False), 'cluster'] = 12
print('12 finished')
product_information.loc[product_information['name'].str.contains('|'.join(health), case=False, na=False), 'cluster'] = 13
print('13 finished')
product_information.loc[product_information['name'].str.contains('|'.join(sauces), case=False, na=False), 'cluster'] = 14
print('14 finished')
product_information.loc[product_information['name'].str.contains('|'.join(gadgets), case=False, na=False), 'cluster'] = 15
print('15 finished')
product_information.loc[product_information['name'].str.contains('для', case=False, na=False), 'cluster'] = 16
print('16 finished')
product_information.loc[product_information['name'].str.contains('цвет', case=False, na=False), 'cluster'] = 17
print('17 finished')
product_information.loc[product_information['name'].str.contains('|'.join(spices), case=False, na=False), 'cluster'] = 18
print('18 finished')
product_information.loc[product_information['name'].str.contains('|'.join(woman), case=False, na=False), 'cluster'] = 19
print('19 finished')
product_information.loc[product_information['name'].str.contains('|'.join(man), case=False, na=False), 'cluster'] = 20
print('20 finished')


remaining_negatives = (product_information['cluster'] == -1).sum()
print(f"Осталось строк с меткой -1: {remaining_negatives}")

1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
Осталось строк с меткой -1: 16408


In [49]:
product_information[product_information['cluster'] == -1]

,product_id,name,brand,type,category_id,category_name,price,discount_price,cluster
0,26176363,Развивающие тесты (3-4 года) (нов.обл.) | Земц...,Machaon,Печатная книга: Развитие детей,780,Книги,380.0,274.0,-1
15,146808464,Набор игровой Плей-До Тостер PLAY-DOH E0039,Hasbro,Набор для лепки,385,Лепка,2155.0,1058.0,-1
34,180734013,"Аккумулятор Greenworks 2918407, 60V, 4 Ач",Greenworks,Аккумулятор для инструмента,320,Строительство и ремонт,16740.0,13392.0,-1
39,200860750,Справочник оттенков RAL Classic K7,RAL,Каталог отделочных материалов,320,Строительство и ремонт,1550.0,1550.0,-1
51,232490092,"Автомобильное зарядное предпусковое АКБ (4,7-1...",НПП Орион,Устройство пуско-зарядное,193,Аксессуары для автомобиля,6855.0,4686.0,-1
...,...,...,...,...,...,...,...,...,...
238324,592203531,Щипцы кухонные универсальные прочные 34 см Pri...,Prime Chef,Щипцы кухонные,571,Кухонные принадлежности,1990.0,313.0,-1
238334,668728163,Фигурка Геншин Импакт Кли / Genshin Impact (16см),Нет бренда,Фигурка,869,Фигурки,999.0,399.0,-1
238345,781502820,"SANC 21.5"" Монитор N2453G, черный",SANC,Монитор,269,Мониторы,12999.0,7293.0,-1
238374,1421573833,"Шпикачки Деревенские Дым Дымыч, 500 г",Дым Дымыч,Шпикачки,195,Сардельки,239.0,219.0,-1


In [50]:
product_information.to_csv('cluster_product_information.csv', index=False)

In [51]:
ss = pl.read_csv(
    os.path.join(data_path, 'cluster_product_information.csv'),
    ignore_errors=True
)